## 데이터

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Load the training data
train_data_path = 'C:/Users/82106/Desktop/데이터분석 프로젝트/서울시 평균 기온 예측/data/open/train.csv'
train_data = pd.read_csv(train_data_path)

## 결측치 처리

In [2]:
# Check for missing values in the training data
missing_values = train_data.isnull().sum()
missing_values_percentage = (missing_values / len(train_data)) * 100

missing_values_info = pd.DataFrame({
    'Missing Values': missing_values, 
    'Percentage (%)': missing_values_percentage
})

missing_values_info[missing_values_info['Missing Values'] > 0]


,Missing Values,Percentage (%)
최고기온,3,0.013037
최저기온,3,0.013037
일교차,4,0.017383
강수량,13861,60.236409
평균풍속,4,0.017383
일조합,118,0.512798
일사합,4862,21.129025
일조율,366,1.590544


In [3]:
# Filling missing values
# For precipitation (강수량), assume 0 where data is missing
# For other features with very few missing values, use median to fill the gaps

train_data['강수량'].fillna(0, inplace=True)
train_data['일조합'].fillna(0, inplace=True)
train_data['일조율'].fillna(0, inplace=True)

# For other features, using median
features_to_fill_median = ['최고기온', '최저기온', '일교차', '평균풍속', '일사합']
train_data[features_to_fill_median] = train_data[features_to_fill_median].fillna(train_data[features_to_fill_median].median())

# Check if there are any missing values left
remaining_missing_values = train_data.isnull().sum().sum()

remaining_missing_values


0

## 범주형 원핫인코딩

In [4]:
# 데이터프레임의 열 데이터 유형 확인
data_types = train_data.dtypes

# 범주형 열을 저장할 리스트 초기화
categorical_columns = []

# 열 데이터 유형이 object인 열을 찾아 범주형 열로 판단
for column, dtype in data_types.items():
    if dtype == 'object':
        categorical_columns.append(column)

# 범주형 열 목록 출력
print("범주형 열 목록:", categorical_columns)


범주형 열 목록: ['일시']


In [5]:
# 범주형 열을 원-핫 인코딩으로 변환
train_data = pd.get_dummies(train_data, columns=['일시'])

# 원-핫 인코딩된 데이터 확인
print(train_data.head())

   최고기온  최저기온   일교차  강수량  평균습도  평균풍속  일조합    일사합  일조율  평균기온  ...  \
0   2.2  -5.2   7.4  0.0  68.3   1.7  6.7  11.22  0.0  -1.6  ...   
1   1.2  -5.6   6.8  0.4  87.7   1.3  0.0  11.22  0.0  -1.9  ...   
2   8.7  -2.1  10.8  0.0  81.3   3.0  0.0  11.22  0.0   4.0  ...   
3  10.8   1.2   9.6  0.0  79.7   4.4  2.6  11.22  0.0   7.5  ...   
4   1.3  -8.2   9.5  0.0  44.0   5.1  8.2  11.22  0.0  -4.6  ...   

   일시_2022-12-22  일시_2022-12-23  일시_2022-12-24  일시_2022-12-25  일시_2022-12-26  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   일시_2022-12-27  일시_2022-12-28  일시_2022-12-29  일시_2022-12-30  일시_2022-12-31  
0              0              0

## 스케일링

In [6]:
# 열별 데이터의 최솟값과 최댓값 확인
column_min = train_data.min()
column_max = train_data.max()

# 열의 범위 계산
column_range = column_max - column_min

# 열의 범위가 큰 열을 출력
scaling_required_columns = column_range[column_range > 1.0]
print("스케일링이 필요한 열:", scaling_required_columns.index.tolist())


스케일링이 필요한 열: ['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율', '평균기온']


In [7]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일러 생성
scaler = MinMaxScaler()

# 스케일링할 열 선택
columns_to_scale = ['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율', '평균기온']  # 스케일링할 열 이름을 나열

# 선택한 열에 Min-Max 스케일링 적용
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])

# 스케일링된 데이터 확인
print(train_data.head())


       최고기온      최저기온       일교차       강수량      평균습도      평균풍속       일조합  \
0  0.296992  0.297030  0.344086  0.000000  0.615385  0.207792  0.489051   
1  0.278195  0.289109  0.311828  0.001202  0.852259  0.155844  0.000000   
2  0.419173  0.358416  0.526882  0.000000  0.774115  0.376623  0.000000   
3  0.458647  0.423762  0.462366  0.000000  0.754579  0.558442  0.189781   
4  0.280075  0.237624  0.456989  0.000000  0.318681  0.649351  0.598540   

        일사합  일조율      평균기온  ...  일시_2022-12-22  일시_2022-12-23  일시_2022-12-24  \
0  0.335125  0.0  0.295409  ...              0              0              0   
1  0.335125  0.0  0.289421  ...              0              0              0   
2  0.335125  0.0  0.407186  ...              0              0              0   
3  0.335125  0.0  0.477046  ...              0              0              0   
4  0.335125  0.0  0.235529  ...              0              0              0   

   일시_2022-12-25  일시_2022-12-26  일시_2022-12-27  일시_2022-12-28  일시_20

## 타겟변수 분리

In [11]:
# 입력 특성과 타겟 변수 분리
X = train_data.drop(columns=['평균기온'])
y = train_data['평균기온']


## 특성 선택

In [12]:
from sklearn.feature_selection import SelectKBest, f_regression
# 특성 선택: F-검정을 사용하여 상위 k개의 특성 선택 (k는 선택할 특성의 개수)
k = 5
selector = SelectKBest(score_func=f_regression, k=k)
X_new = selector.fit_transform(X, y)


In [13]:
selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]


In [14]:
selected_features

Index(['최고기온', '최저기온', '평균습도', '일사합', '일조율'], dtype='object')

In [16]:
from sklearn.model_selection import train_test_split
# 입력 특성과 타겟 변수 분리
X = train_data[['최고기온', '최저기온', '평균습도', '일사합', '일조율']]
y = train_data['평균기온']

# 데이터를 학습용과 테스트용으로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
# 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 학습된 모델로 예측
y_pred = model.predict(X_test)

In [20]:
y_pred

array([0.59133634, 0.790784  , 0.85254843, ..., 0.49046825, 0.30480297,
       0.54718118])

In [19]:
from sklearn.metrics import mean_squared_error, r2_score
# 모델 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"평균 제곱 오차 (MSE): {mse:.2f}")
print(f"결정 계수 (R-squared): {r2:.2f}")

평균 제곱 오차 (MSE): 0.00
결정 계수 (R-squared): 1.00


In [27]:
import pandas as pd

# sample_submission.csv 파일을 읽어옵니다.
sample_submission = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/서울시 평균 기온 예측/data/open/sample_submission.csv')

# train 데이터의 마지막 날짜 다음 날짜부터 sample_submission에 있는 날짜까지를 예측합니다.
start_date = '2023-01-01'
end_date = '2023-12-24'
date_range = pd.date_range(start=start_date, end=end_date)

# 해당 날짜 범위로 예측한 평균 기온 값을 가져와서 sample_submission에 할당합니다.
sample_submission['평균기온'] = y_pred[:len(date_range)]

# 결과를 새로운 CSV 파일로 저장합니다.
sample_submission.to_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/서울시 평균 기온 예측/data/open/submission.csv', index=False)


## SVR

In [29]:
from sklearn.svm import SVR
svm_model = SVR(kernel='linear')  # You can change the kernel type as needed
svm_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_svm = svm_model.predict(X_test)


In [30]:
# Evaluate the model's performance
mse_svm = mean_squared_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print(f"SVM - MSE: {mse_svm:.4f}, R-squared: {r2_svm:.4f}")

SVM - MSE: 0.0010, R-squared: 0.9778


In [32]:
import pandas as pd

# sample_submission.csv 파일을 읽어옵니다.
sample_submission = pd.read_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/서울시 평균 기온 예측/data/open/sample_submission.csv')

# train 데이터의 마지막 날짜 다음 날짜부터 sample_submission에 있는 날짜까지를 예측합니다.
start_date = '2023-01-01'
end_date = '2023-12-24'
date_range = pd.date_range(start=start_date, end=end_date)

# 해당 날짜 범위로 예측한 평균 기온 값을 가져와서 sample_submission에 할당합니다.
sample_submission['평균기온'] = y_pred_svm[:len(date_range)]

# 결과를 새로운 CSV 파일로 저장합니다.
sample_submission.to_csv('C:/Users/82106/Desktop/데이터분석 프로젝트/서울시 평균 기온 예측/data/open/submission2.csv', index=False)
